# Spectra of Adversarial Examples
Since the gradients that are used to turn an image it into an adversarial example seem to consist mostly of very high frequencies, the spectra of AEs are of interest. To compute and display them, we first need to set up an appropriate network. We start a bunch of imports and by setting some constants.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as ppt
import caffe
import adex
import adex.core
import adex.data
import adex.imagenet

%matplotlib inline
caffe.set_mode_cpu()

caffe_root = '/home/chrisbot/Projects/caffe'
data_root = '/media/sf_Masterarbeit/data'
spectrum_root = data_root

BATCH_LIMIT = 1 # TODO set to None
BATCH_SIZE = 1 # how many images to process at once
CONFIDENCE_TARGET = 0.9
AE_GRADIENT_COEFFICIENT = 0.9
AE_GENERATION_ITERATIONS = 10

Next, we load the network model, the labels and a transformer that prepares our images to fit inside the network's first layer.

In [2]:
net = adex.imagenet.load_model(caffe_root, BATCH_SIZE)
imagenet_labels = adex.imagenet.load_labels(caffe_root)
transformer = adex.imagenet.build_transformer(net)

To conclude preparation, we need data.

In [3]:
image_list_file = data_root + '/ILSVRC2012_img_train_t3.txt'

with open(image_list_file) as f:
    image_list = f.readlines()
image_list = [line.strip() for line in image_list]

data_shape = net.blobs['data'].data.shape

## Inspecting the spectra
At last, we compute the spectrum of the data by computing each image's spectrum and summing the result. For each image we generate an adversarial example and do the same.

In [4]:
original_spectrum = np.zeros(shape=(1, 1, data_shape[2], data_shape[3]))
adversarial_spectrum = np.zeros_like(original_spectrum)
#for _ in range(BATCH_SIZE):
#    image_list.insert(0, 'example_images/panda.jpg')
for i in xrange(0, len(image_list), BATCH_SIZE): # Step through the data in chunks of BATCH_SIZE
    if BATCH_LIMIT is not None and i >= BATCH_LIMIT:
        break
    
    # The images for this batch. Stop if not enough.
    batch_list = image_list[i:i+BATCH_SIZE]
    if len(batch_list) != BATCH_SIZE:
        break

    # Fetch images from disk
    data = np.empty(data_shape)
    for index, image in enumerate(batch_list): # Load BATCH_SIZE many images
        image_data = caffe.io.load_image(data_root + '/' + image)
        data[index,...] = transformer.preprocess('data', image_data)
    
    # Generate adversarial examples
    #desired_labels = np.random.randint(len(imagenet_labels), size=data.shape[0]) #FIXME
    desired_labels = np.full(shape=(BATCH_SIZE), fill_value=23, dtype=np.int64)
    adversarial_data, confidence, iterations = adex.core.make_adversarial(net, data, desired_labels,
                                                  AE_GRADIENT_COEFFICIENT, CONFIDENCE_TARGET, AE_GENERATION_ITERATIONS)
    
    # Get spectra
    original_spectrum += np.sum(adex.data.get_spectrum(adex.data.grayvalue_image(data)),
                                axis=0, keepdims=True)
    adversarial_spectrum += np.sum(adex.data.get_spectrum(adex.data.grayvalue_image(adversarial_data)),
                                   axis=0, keepdims=True)

average confidence of 2.75687199291e-07 after iteration 0
average confidence of 1.33865887619e-06 after iteration 1
average confidence of 5.67068900637e-06 after iteration 2
average confidence of 2.04733587452e-05 after iteration 3
average confidence of 6.52068993077e-05 after iteration 4
average confidence of 0.000173987034941 after iteration 5
average confidence of 0.000405439845053 after iteration 6
average confidence of 0.000855442136526 after iteration 7
average confidence of 0.00161580229178 after iteration 8
average confidence of 0.00280778110027 after iteration 9


Lastly, the images are and saved to disk.

In [ ]:
# Save data to disk
np.save(spectrum_root + 'original_spectrum.npy', original_spectrum)
np.save(spectrum_root + 'adversarial_spectrum.npy', adversarial_spectrum)

## Visualizing the results
The following part is meant to run independently. It loads the data from disk and visualizes it.

In [ ]:
original_spectrum = np.load(spectrum_root + 'original_spectrum.npy')
adversarial_spectrum = np.load(spectrum_root + 'adversarial_spectrum.npy')
log_spectrum_difference = np.log(original_spectrum) - np.log(adversarial_spectrum)

def activity_distribution(spectrum, bins):
    spectrum = spectrum.reshape(spectrum.shape[-2:])

    # Record the summed values at every distance to the origin, and the number of summed values
    distribution = np.zeros(shape=(bins))
    pixel_count = np.zeros_like(distribution)
    
    center = np.asarray(spectrum.shape) / 2.0
    max_distance = math.ceil(np.linalg.norm(center))
    
    it = np.nditer(spectrum, flags=['multi_index'])
    for value in it:
        distance = np.linalg.norm(np.asarray(it.multi_index) - center)
        index = (distance / max_distance) * (bins - 1)
        index = int(round(index))
        
        distribution[index] += value
        pixel_count[index] += 1
    
    return distribution / (pixel_count + 1)

# Plot the distribution of frequencies over distance from the origin on a log scale
original_freq_distribution = activity_distribution(original_spectrum, 256)
adversarial_freq_distribution = activity_distribution(adversarial_spectrum, 256)
log_spectrum_difference_freq_distribution = activity_distribution(log_spectrum_difference, 256)

ppt.figure(figsize=(16, 4))

ax = ppt.subplot(1, 3, 1)
ax.set_xscale('log')
ax.set_yscale('log')
ppt.plot(original_freq_distribution)
ppt.title('Original')

ax = ppt.subplot(1, 3, 2)
ax.set_xscale('log')
ax.set_yscale('log')
ppt.plot(adversarial_freq_distribution)
ppt.title('Adversarial')


ax = ppt.subplot(1, 3, 3)
ax.set_xscale('log')
#ax.set_yscale('log')
ppt.plot(log_spectrum_difference_freq_distribution)
ppt.title('Log-spectrum difference')


# Display images
try:
    ppt.figure(figsize=(16, 4))
    
    ppt.subplot(1, 3, 1)
    ppt.imshow(np.log(original_spectrum[0,0]), cmap='gray', interpolation='none')
    ppt.title('Original')
    
    ppt.subplot(1, 3, 2)
    ppt.imshow(np.log(adversarial_spectrum[0,0]), cmap='gray', interpolation='none')
    ppt.title('Adversarial')
    
    ppt.subplot(1, 3, 3)
    ppt.imshow(log_spectrum_difference[0,0], cmap='gray', interpolation='none')
    ppt.title('Spectrum difference')
except:
    print('Could not display image')